In [1]:
import sys 
sys.path.append('..')

In [2]:
import numpy as np
import pandas as pd

from preprocessing import Preprocessing
from sklearn.model_selection import StratifiedKFold, GridSearchCV

from xgboost import XGBClassifier

In [3]:
col_drop = ['CustomerId','Surname']
types = {
    'HasCrCard':bool,
    'IsActiveMember':bool
}
train = pd.read_csv('../data/train.csv', index_col='id', dtype=types).drop(columns=col_drop)
train.drop_duplicates(inplace=True)
X = train.drop(columns='Exited')
y = train.Exited

In [4]:
prepro = Preprocessing()
X_scld = prepro.fit_transform(X).astype(float)
X_scld.shape

(164911, 91)

In [5]:
skf = StratifiedKFold(n_splits=4, shuffle=True)
WeightTarget = y.value_counts(True).to_dict()

In [6]:
params = {
    'booster'                : ['dart'],
    'objective'              : ['binary:logistic'],
    'learning_rate'          : [1e-1, 1e-2],
    'n_estimators'           : [50, 100],
    'max_depth'              : [4, 8, 16],
    'subsample'              : [0.8, 0.9, 1.0],
    'eval_metric'            : ['auc'],
    'rate_drop'              : [0.1, 0.2],
    'skip_drop'              : [0.1, 0.2],
}
model = XGBClassifier()
grid = GridSearchCV(
    model,
    params,
    scoring = 'roc_auc',
    cv = skf,
    n_jobs = -1,
    verbose=20
)
grid.fit(X_scld,y)

Fitting 4 folds for each of 144 candidates, totalling 576 fits
[CV 1/4; 1/144] START booster=dart, eval_metric=auc, learning_rate=0.1, max_depth=4, n_estimators=50, objective=binary:logistic, rate_drop=0.1, skip_drop=0.1, subsample=0.8
[CV 2/4; 1/144] START booster=dart, eval_metric=auc, learning_rate=0.1, max_depth=4, n_estimators=50, objective=binary:logistic, rate_drop=0.1, skip_drop=0.1, subsample=0.8
[CV 4/4; 1/144] START booster=dart, eval_metric=auc, learning_rate=0.1, max_depth=4, n_estimators=50, objective=binary:logistic, rate_drop=0.1, skip_drop=0.1, subsample=0.8
[CV 1/4; 2/144] START booster=dart, eval_metric=auc, learning_rate=0.1, max_depth=4, n_estimators=50, objective=binary:logistic, rate_drop=0.1, skip_drop=0.1, subsample=0.9
[CV 3/4; 2/144] START booster=dart, eval_metric=auc, learning_rate=0.1, max_depth=4, n_estimators=50, objective=binary:logistic, rate_drop=0.1, skip_drop=0.1, subsample=0.9
[CV 3/4; 1/144] START booster=dart, eval_metric=auc, learning_rate=0.1, 

KeyboardInterrupt: 

In [ ]:
grid.best_params_, grid.best_score_

In [ ]:
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score').head(10)